<a href="https://colab.research.google.com/github/FewchX/Predict_health_costs_with_regression/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.head(25)
#print(dataset.shape)

In [ ]:
# Preprocess the data
expenses = dataset['expenses']
features = dataset.drop(columns=['expenses'])

# Convert categorical features to numerical
features['sex'] = features['sex'].apply(lambda x: 0 if x == 'male' else 1)
features['smoker'] = features['smoker'].apply(lambda x: 1 if x == 'yes' else 0)
region_mapping = {'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3}
features['region'] = features['region'].map(region_mapping)


# Split the data into training and test sets
train_size = int(0.8 * len(dataset))

X_train = features.iloc[:train_size]
X_test = features.iloc[train_size:]
y_train = expenses.iloc[:train_size]
y_test = expenses.iloc[train_size:]

In [ ]:
# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the model
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    validation_split=0.2,
    callbacks=[tfdocs.modeling.EpochDots()]
)

# Evaluate the model
#loss, mae = model.evaluate(X_test, y_test)
#print(f"Test set Mean Absolute Error: {mae:.2f}")

In [ ]:
loss, mae = model.evaluate(X_train, y_train, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
